In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core.tools import FunctionTool
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()

api_version = os.getenv("AZURE_OPENAI_API_VERSION")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
azure_openai_chat_completion_client = AzureOpenAIChatCompletionClient(
            model=deployment_name,
            azure_endpoint=azure_endpoint,
            api_version=api_version,
            api_key=api_key,
        )


In [2]:
def google_search(query: str, num_results: int = 2, max_chars: int = 500) -> list:  # type: ignore[type-arg]
    import os
    import time

    import requests
    from bs4 import BeautifulSoup
    from dotenv import load_dotenv

    load_dotenv()

    serp_api_key = os.getenv("SERPAPI_KEY")

    if not serp_api_key:
        raise ValueError("SerpAPI key not found in environment variables")

    url = "https://serpapi.com/search"
    params = {
        "engine": "google",
        "q": query,
        "num": num_results,
        "api_key": serp_api_key,
    }

    response = requests.get(url, params=params)

    if response.status_code != 200:
        print(response.json())
        raise Exception(f"Error in API request: {response.status_code}")

    results = response.json().get("organic_results", [])

    def get_page_content(url: str) -> str:
        try:
            response = requests.get(url, timeout=10)
            soup = BeautifulSoup(response.content, "html.parser")
            text = soup.get_text(separator=" ", strip=True)
            words = text.split()
            content = ""
            for word in words:
                if len(content) + len(word) + 1 > max_chars:
                    break
                content += " " + word
            return content.strip()
        except Exception as e:
            print(f"Error fetching {url}: {str(e)}")
            return ""

    enriched_results = []
    for item in results:
        body = get_page_content(item["link"])
        enriched_results.append(
            {"title": item["title"], "link": item["link"], "snippet": item.get("snippet", ""), "body": body}
        )
        time.sleep(1)  # Be respectful to the servers

    print(enriched_results)    

    return enriched_results


In [3]:
def arxiv_search(query: str, max_results: int = 2) -> list:  # type: ignore[type-arg]
    """
    Search Arxiv for papers and return the results including abstracts.
    """
    import arxiv

    client = arxiv.Client()
    search = arxiv.Search(query=query, max_results=max_results, sort_by=arxiv.SortCriterion.Relevance)

    results = []
    for paper in client.results(search):
        results.append(
            {
                "title": paper.title,
                "authors": [author.name for author in paper.authors],
                "published": paper.published.strftime("%Y-%m-%d"),
                "abstract": paper.summary,
                "pdf_url": paper.pdf_url,
            }
        )


    print(results)
    return results

In [4]:
google_search_tool = FunctionTool(
    google_search, description="Search Google for information, returns results with a snippet and body content"
)
arxiv_search_tool = FunctionTool(
    arxiv_search, description="Search Arxiv for papers related to a given topic, including abstracts"
)

In [5]:
google_search_agent = AssistantAgent(
    name="Google_Search_Agent",
    tools=[google_search_tool],
    model_client=azure_openai_chat_completion_client,
    description="An agent that can search Google for information, returns results with a snippet and body content",
    system_message="You are a helpful AI assistant. Solve tasks using your tools.",
)

arxiv_search_agent = AssistantAgent(
    name="Arxiv_Search_Agent",
    tools=[arxiv_search_tool],
    model_client=azure_openai_chat_completion_client,
    description="An agent that can search Arxiv for papers related to a given topic, including abstracts",
    system_message="You are a helpful AI assistant. Solve tasks using your tools. Specifically, you can take into consideration the user's request and craft a search query that is most likely to return relevant academi papers.",
)


report_agent = AssistantAgent(
    name="Report_Agent",
    model_client=azure_openai_chat_completion_client,
    description="Generate a report based on a given topic",
    system_message="You are a helpful assistant. Your task is to synthesize data extracted into a high quality literature review including CORRECT references. You MUST write a final report that is formatted as a literature review with CORRECT references.  Your response should end with the word 'TERMINATE'",
)

In [6]:
termination = TextMentionTermination("TERMINATE")
team = RoundRobinGroupChat(
    participants=[google_search_agent, arxiv_search_agent, report_agent], termination_condition=termination
)

In [7]:
await Console(
    team.run_stream(
        task="멀티 에이전트 AI 시스템 구축을 위한한 논문 리뷰 리포트 작성",
        # task="최신 RAG 기술에 대한 동향 및 논문 리뷰",
    )
)

---------- TextMessage (user) ----------
멀티 에이전트 AI 시스템 구축을 위한한 논문 리뷰 리포트 작성
---------- ThoughtEvent (Google_Search_Agent) ----------
멀티 에이전트 AI 시스템 구축을 위한 논문 리뷰 리포트를 작성하기 위해, 관련 최신 논문들을 조사하고 주요 내용과 기여점을 분석하는 과정이 필요합니다. 이를 위해 우선 멀티 에이전트 AI 시스템과 관련된 최신 연구 동향을 살펴보고, 대표적인 논문들을 선정하여 리뷰하는 작업을 진행하겠습니다.

관련 최신 논문들을 검색하여 주요 논문들을 선정 후, 각 논문의 핵심 내용, 사용된 방법론, 실험 결과 및 한계점 등을 포함하는 리뷰 리포트를 작성할 수 있도록 하겠습니다.

잠시만 기다려 주세요.
---------- ToolCallRequestEvent (Google_Search_Agent) ----------
[FunctionCall(id='call_QY8DeNlqdgKQGObXtXjXLi7x', arguments='{"query":"multi-agent AI system research papers review","num_results":3}', name='google_search')]
[{'title': 'Combining Multi-Agent Systems and Artificial Intelligence ...', 'link': 'https://www.sciencedirect.com/science/article/abs/pii/S2542660524003056', 'snippet': 'A Multi-Agent System (MAS) usually refers to a network of autonomous agents that interact with each other to achieve a common objective.', 'body': 'ScienceDirect Skip to main content ScienceDirec

TaskResult(messages=[TextMessage(id='8a4cea62-4ee2-48c1-8b99-717009b8f629', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 15, 6, 49, 3, 838022, tzinfo=datetime.timezone.utc), content='멀티 에이전트 AI 시스템 구축을 위한한 논문 리뷰 리포트 작성', type='TextMessage'), ThoughtEvent(id='19a322a5-5725-46ad-88fd-9411237a8633', source='Google_Search_Agent', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 15, 6, 49, 7, 192980, tzinfo=datetime.timezone.utc), content='멀티 에이전트 AI 시스템 구축을 위한 논문 리뷰 리포트를 작성하기 위해, 관련 최신 논문들을 조사하고 주요 내용과 기여점을 분석하는 과정이 필요합니다. 이를 위해 우선 멀티 에이전트 AI 시스템과 관련된 최신 연구 동향을 살펴보고, 대표적인 논문들을 선정하여 리뷰하는 작업을 진행하겠습니다.\n\n관련 최신 논문들을 검색하여 주요 논문들을 선정 후, 각 논문의 핵심 내용, 사용된 방법론, 실험 결과 및 한계점 등을 포함하는 리뷰 리포트를 작성할 수 있도록 하겠습니다.\n\n잠시만 기다려 주세요.', type='ThoughtEvent'), ToolCallRequestEvent(id='1a49ecda-18f0-4653-80b0-c9e0b84e040d', source='Google_Search_Agent', models_usage=RequestUsage(prompt_tokens=128, completion_tokens=163), metadata={}, created_at=datetime.